# ILD CNN prediction part
## data set is testset
    the model is rebuilt from the JSON file and recompiled with the weights 
    the weights are loaded from ILD_CNN_model_weights
    

In [1]:
import argparse
import numpy as np
import sklearn.metrics as metrics
import cPickle as pickle
import json
from keras.models import model_from_json
import sys
import cv2

from keras.models import Sequential
from keras.utils import np_utils 
from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D,AveragePooling2D
from keras.layers.advanced_activations import LeakyReLU

Using Theano backend.


In [2]:
# hinting the path into the python directory where my helper and model files reside
sys.path.insert(0, '../python')

In [3]:
import ild_helpers as H
import cnn_model as CNN

In [4]:
args         = H.parse_args()                          
train_params = {
     'do' : float(args.do) if args.do else 0.5,        
     'a'  : float(args.a) if args.a else 0.3,          # Conv Layers LeakyReLU alpha param [if alpha set to 0 LeakyReLU is equivalent with ReLU]
     'k'  : int(args.k) if args.k else 4,              # Feature maps k multiplier
     's'  : float(args.s) if args.s else 1,            # Input Image rescale factor
     'pf' : float(args.pf) if args.pf else 1,          # Percentage of the pooling layer: [0,1]
     'pt' : args.pt if args.pt else 'Avg',             # Pooling type: Avg, Max
     'fp' : args.fp if args.fp else 'proportional',    # Feature maps policy: proportional, static
     'cl' : int(args.cl) if args.cl else 5,            # Number of Convolutional Layers
     'opt': args.opt if args.opt else 'Adam',          # Optimizer: SGD, Adagrad, Adam
     'obj': args.obj if args.obj else 'ce',            # Minimization Objective: mse, ce
     'patience' : args.pat if args.pat else 5,         # Patience parameter for early stoping
     'tolerance': args.tol if args.tol else 1.005,     # Tolerance parameter for early stoping [default: 1.005, checks if > 0.5%]
     'res_alias': args.csv if args.csv else 'res'      # csv results filename alias
}

In [5]:
# load both the model and the weights

model = H.load_model()

In [6]:
model.compile(optimizer='Adam', loss=CNN.get_Obj(train_params['obj']))

In [7]:
X_predict = pickle.load( open( "../pickle/X_test.pkl", "rb" ) )

In [8]:
# adding a singleton dimension and rescale to [0,1]
X_predict = np.asarray(np.expand_dims(X_predict,1))/float(255)

In [9]:
# predict and check classification and probabilities are the same

classes = model.predict_classes(X_predict, batch_size=10)
proba = model.predict_proba(X_predict, batch_size=10)

6250/6250 [==============================] - 70s    


In [10]:
classes

array([4, 1, 4, ..., 2, 1, 1])

In [11]:
proba

array([[  8.80201063e-15,   2.21276838e-12,   1.83340744e-19,
          4.70158579e-10,   1.00000000e+00,   3.61959615e-21],
       [  1.67203951e-04,   6.61034942e-01,   3.23322296e-01,
          5.70375869e-06,   7.06237834e-03,   8.40748660e-03],
       [  5.33210073e-11,   2.05621090e-10,   2.36306547e-11,
          1.16759853e-03,   9.98832405e-01,   1.20484999e-14],
       ..., 
       [  1.85666261e-13,   2.85946587e-13,   7.36005843e-01,
          2.63994187e-01,   2.11291776e-10,   4.61224652e-14],
       [  3.75343623e-09,   9.98862565e-01,   4.75297128e-11,
          5.43771842e-12,   2.02445462e-05,   1.11716636e-03],
       [  6.03482431e-06,   9.99972880e-01,   1.95290522e-05,
          3.20099417e-07,   1.54640500e-09,   1.26309692e-06]])

In [13]:
pickle.dump(classes, open( "../pickle/predicted_classes_test_set.pkl", "wb" ))
pickle.dump(proba, open( "../pickle/predicted_probabilities_test_set.pkl", "wb" ))